<a href="https://colab.research.google.com/github/dennisbarchetti/natural_language_processsing/blob/master/Ricerca_sinonimi_aggettivi_pi%C3%B9_frequenti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
!pip install spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 4.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [109]:
import nltk

import spacy #Modulo per elaborazione testo naturale (nlp)
from collections import Counter #modulo per contare (verbi/pronomi/nomi...)
import pandas as pd #database ricco di dati (già filtrati/elaborati)
import numpy as np;
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)

from nltk.corpus import wordnet

In [110]:
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package dependency_treebank is already up-to-date!
[nltk_data]    | Downloadi

True

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
import os
os.listdir("/content/drive/MyDrive/Colab_Notebooks")
infile = open("/content/drive/MyDrive/Colab_Notebooks/Testo.txt")
sample_text = infile.read()

MessageError: ignored

ricerco tutti gli aggettivi presenti nel testo e li inserisco su un dataframe ordinati per numero di apparizioni 

In [111]:
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
# Process whole documents
text = ("""We shall be Conservative, in all cases where we think Conservatism essential to the public good;—and we shall be Radical in everything which may seem to us to require radical treatment and radical reform. 
          We do not believe that everything in Society is either exactly right or exactly wrong;—what is good we desire to preserve and improve;—what is evil, to exterminate, or reform.""")
  
doc = nlp(text)
  
# Token and Tag
for token in doc: 
  #print(token, token.pos_)

 aggettivi = []
for token in doc:
    if token.pos_ == "ADJ":
        #print(token, token.pos_)
        aggettivi.append(token.text) 
print(aggettivi)

numero_aggettivi= Counter(aggettivi)
dfA = pd.DataFrame(numero_aggettivi.most_common(),columns=['aggettivi', 'count']) 
dfA[:100] #primi 100 elementi del dataframe


['Conservative', 'essential', 'radical', 'radical', 'right', 'good', 'evil']


,aggettivi,count
0,radical,2
1,Conservative,1
2,essential,1
3,right,1
4,good,1
5,evil,1


inserisco in una lista i 3 aggettivi ripetuti più volte nel testo

In [112]:
tre_ADJ_mostcommon = []
tre_ADJ_mostcommon.append(numero_aggettivi.most_common(3))

#print(tre_ADJ_mostcommon)  #tre_ADJ_mostcommon è una lista coplessa a me interessa solo la parola e non il numero di volte che appare
#print(tre_ADJ_mostcommon[0][0][0])

listaADJ = [] # lista di parole filtrate da altre informazioni che non mi servono
i=0
while i< len(tre_ADJ_mostcommon[0]):
  listaADJ.append(tre_ADJ_mostcommon[0][i][0]) 
  i=i+1
print(listaADJ)

['radical', 'Conservative', 'essential']


ricerco i sinonimi per ognuno dei 3 aggettivi, stampo una lista di sinonimi per ognono di essi con anche il numero di elementi di ognuna 

Successivamente li inserisco in 3 liste attraverso le quali vado a creaare dei dataframe

In [113]:
list1 = []
list2 = []
list2 = []

i = 0
for aggettivo in listaADJ:
  sentence = aggettivo
  tokens = nltk.word_tokenize(sentence)
  synonms = set()
  for token in tokens:
    synsets= wordnet.synsets(token)
    for syn in synsets:
      for lemma in syn.lemmas():
        synonms.add(lemma.name())
  numero_sinonimi = 0
  for sinonimo in synonms:
    numero_sinonimi= numero_sinonimi+1
  #print("sinonimi di " + aggettivo)
  #print(synonms)
  #print(numero_sinonimi)
  if i == 0:
    list1 = synonms
  if i == 1:
    list2 = synonms 
  if i == 2:
    list3 = synonms 
  i = i + 1

print(list1)
print(list2)
print(list3)

i = 0
for aggettivo in listaADJ:
  if i == 0:
     df1 = pd.DataFrame(list1,columns=[aggettivo]) 
  if i == 1:
    df2 = pd.DataFrame(list2,columns=[aggettivo])
  if i == 2:
    df3 = pd.DataFrame(list3,columns=[aggettivo]) 
  i = i + 1

dfu = df1.combine_first(df2)
dfu = dfu.combine_first(df3)
dfu[:100] #primi 100 elementi del dataframe

# altre possibilità che però non funzionano
# df1.append(df2)
# df1.merge(df2, left_on=list1, right_on=list2)
# df1.join(...)


{'stem', 'root_word', 'group', 'ultra', 'revolutionary', 'radical', 'extremist', 'theme', 'basal', 'chemical_group', 'base', 'root', 'free_radical'}
{'conservativist', 'cautious', 'button-down', 'materialistic', 'Conservative', 'buttoned-down', 'bourgeois', 'conservative'}
{'necessary', 'crucial', 'all_important', 'of_the_essence', 'requisite', 'requirement', 'necessity', 'essential', 'substantive', 'all-important', 'indispensable'}


,Conservative,essential,radical
0,conservativist,necessary,stem
1,cautious,crucial,root_word
2,button-down,all_important,group
3,materialistic,of_the_essence,ultra
4,Conservative,requisite,revolutionary
5,buttoned-down,requirement,radical
6,bourgeois,necessity,extremist
7,conservative,essential,theme
8,NaN,substantive,basal
9,NaN,all-important,chemical_group


In [ ]:
df1[:100] #primi 100 elementi del dataframe

,radical
0,stem
1,root_word
2,group
3,ultra
4,revolutionary
5,radical
6,extremist
7,theme
8,basal
9,chemical_group


In [ ]:
df2[:100] #primi 100 elementi del dataframe

,Conservative
0,conservativist
1,cautious
2,button-down
3,materialistic
4,Conservative
5,buttoned-down
6,bourgeois
7,conservative


In [ ]:
df3[:100] #primi 100 elementi del dataframe

,essential
0,necessary
1,crucial
2,all_important
3,of_the_essence
4,requisite
5,requirement
6,necessity
7,essential
8,substantive
9,all-important
